<a href="https://colab.research.google.com/github/RuslanMavlitov/sf_data_science/blob/main/implicit_%26_LightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!pip install implicit
#!pip install lightfm

## 1. Матричная факторизация и факторизационные машины. Практика

In [8]:
import numpy as np
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

from lightfm import LightFM
from lightfm.evaluation import precision_at_k

Загрузим наш датасет, добавим имена столбцам, затем отсортируем записи по дате и преобразуем целевую переменную в бинарный вид: условимся, что фильмы, которым пользователь поставил оценку выше 2, считаются понравившимися ему (класс 1), в противном случае — не понравившимися (класс 0)

In [6]:
ratings = pd.read_csv("/content/drive/MyDrive/SF/data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)

Затем разделим нашу выборку на тренировочную и тестовую в соотношении 80/20 без перемешивания. В тренировочную выборку попадут оценки пользователей за первые 80 % периода проведения наблюдений, а в тестовую — оставшиеся 20 %

In [9]:
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

Чтобы обучить ALS-модель на предоставленных данных, нужно создать user-item таблицу для тренировочной и тестовой выборки. В этой таблице по строкам должны быть отложены идентификаторы всех уникальных пользователей, которые у нас есть, а по столбцам — все уникальные фильмы. То есть мы должны получить две матрицы размерности 943 x 1682. На пересечении строк и столбцов этих матриц должны быть числа, характеризующие наличие положительных и отрицательных оценок пользователей.

**Проблема заключается в том, что в тренировочную и тестовую выборку могли попасть различные пользователи и различные товары. Посмотрим на это, создав сводные таблицы для тренировочной и тестовой выборок.**

In [10]:
train_pivot = pd.pivot_table(
    train,
    index="user_id", 
    columns="item_id", 
    values="score"
)
test_pivot = pd.pivot_table(
    test,
    index="user_id", 
    columns="item_id", 
    values="score"
)

print(train_pivot.shape)
print(test_pivot.shape)

(751, 1616)
(301, 1448)


Теперь создадим сводную таблицу из таблицы rating, заполнив её ячейки нулями. Получим матрицу размером 943 x 1682. Для тех фильмов, которым пользователь выставил оценку значения, будут равны 0, для остальных — пропуску. 

In [11]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Чтобы получить тренировочную и тестовую user-item таблицы, нам осталось только сложить таблицу shell с соответствующими таблицами train_pivot и test_pivot.

Чтобы корректно обрабатывать пропущенные значения, мы трансформируем 1 в 2, а 0 — в 1. Сами пропуски заполняем нулями. В результате у нас получатся две таблицы размером 943 x 1682, в которых на пересечении пользователя и фильма стоит:

0 — если пользователь не оценил данный фильм;

1 — если пользователь оценил фильм отрицательно;

2 — если пользователь оценил фильм положительно.

In [12]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)

train_pivot.head()

(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Финальный шаг предобработки: модели из библиотеки implicit требуют, чтобы user-item матрицы были представлены в виде разреженных матриц. Для получения разреженной матрицы используется функция csr_matrix() из модуля sparse библиотеки scipy

In [13]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

Обучим ALS-модель с 10-ю факторами

In [14]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

Чтобы сформировать рекомендации для конкретного пользователя, можно воспользоваться методом recommend().

Номера столбцов начинаются от 0, а вот идентификаторы фильмов в нашем примере выражаются числами и начинаются от 1. Поэтому необходимо сначала выделить все уникальные идентификаторы товаров в отдельный вектор, а уже затем обращаться к нему по полученным индексам

In [15]:
unique_items = np.array(train_pivot.columns)
user_id = 14
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]
print('Recomendations ids: {}'.format(recomendations_ids))
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations ids: [293 116  99 275 244 287 283 150 125 596]
Recomendations for user 14: [294 117 100 276 245 288 284 151 126 597]


Итак, для пользователя с идентификатором 14 наша рекомендательная система рекомендовала фильмы под идентификаторами [294 117 100 276 245 288 284 151 126 597]. Чтобы понять, что это за фильмы, можно обратиться к таблице movies. 

In [18]:
movies = pd.read_csv('/content/drive/MyDrive/SF/data/ml-100k/movies.zip')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [25]:
movies_for_user14 = []
for i in [294, 117, 100, 276, 245, 288, 284, 151, 126, 597]:
  movies_for_user14.append(movies.iloc[i-1, 1])
movies_for_user14

['Underneath (1995)',
 'Birdcage, The (1996)',
 'Rumble in the Bronx (Hont faan kui) (1995)',
 'Stargate (1994)',
 'Nell (1994)',
 'Star Trek: Generations (1994)',
 "National Lampoon's Senior Trip (1995)",
 'Love & Human Remains (1993)',
 'Addiction, The (1995)',
 'Ghost in the Shell (Kôkaku kidôtai) (1995)']

User14 явно любит фильмы 1995 года

Теперь определим качество модели на всей тестовой выборке, рассчитав precision для топ 10-рекомендуемых фильмов с помощью функции mean_average_precision_at_k() из библиотеки implicit

In [26]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.087


Итак, наша метрика равна 0.087. То есть из 10 рекомендуемых фильмов пользователям в среднем нравятся 8.7 % из них. Это довольно неплохой показатель для такой простой рекомендательной системы.

### Давайте попробуем улучшить качество рекомендаций, воспользовавшись факторизационными машинами

Обучим факторизационные машины с 10-ю факторами, в качестве функции потерь используем logloss, параметр random_state установим в значение 42. Обучение будем производить на 30 итерациях (эпохах)

In [27]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

Чтобы получить сами рекомендации, необходимо умножить эти веса на -1 и отсортировать их индексы по возрастанию веса. Нам нужны будут только индексы 10 фильмов с наибольшим по модулю весом. Обратившись по полученным индексам к списку идентификаторов фильмов, мы получим рекомендации для конкретного пользователя

In [28]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]


In [29]:
movies_for_user14 = []
for i in [50, 294, 258, 100, 181, 288, 286, 1, 300, 121]:
  movies_for_user14.append(movies.iloc[i-1, 1])
movies_for_user14

['Big Green, The (1995)',
 'Underneath (1995)',
 'Pulp Fiction (1994)',
 'Rumble in the Bronx (Hont faan kui) (1995)',
 'Bushwhacked (1995)',
 'Star Trek: Generations (1994)',
 'Tank Girl (1995)',
 'Toy Story (1995)',
 'Double Happiness (1994)',
 'Basketball Diaries, The (1995)']

Пару фильмов совпали даже в рекомендациях, но прнцип смотреть только 1995 остался

Наконец, рассчитаем средний precision для топ 10 рекомендуемых фильмов по всей тестовой выборке. Для этого можно воспользоваться функцией precision_at_k() из библиотеки lightfm. Результатом будет значение precision@k для каждого из пользователей

In [30]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))

Mean Average Precision at 10: 0.32


Итак, наша метрика составила 0.32, что гораздо выше показателя, полученного с помощью ALS. То есть рекомендации факторизационных машин больше подходят нашим пользователям.